In [2]:
import os
import json
import numpy as np
import pandas as pd
import scipy.sparse as spr

목차

- 1. 데이터 처리 (불러오기,추가,수정,변환)
- 2. 플레이리스트로 추천 (협업필터링)

# 데이터 처리

## 불러오기

In [3]:
with open('data/train.json',encoding='utf-8-sig') as f:
    train_dict = json.load(f)
    
with open('data/song_meta.json',encoding='utf-8-sig') as f:
    song_dict = json.load(f)
    
with open('data/genre_gn_all.json',encoding='utf-8-sig') as f:
    genre_dict = json.load(f)
    
train_df = pd.DataFrame.from_dict(train_dict)
song_df = pd.DataFrame.from_dict(song_dict)

## train_df 전처리

### 일반 전처리
- num_songs,num_tags = '중복제거한 노래 태그 개수'

In [4]:
train_df.head(3)

,tags,id,plylst_title,songs,like_cnt,updt_date
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000
2,"[까페, 잔잔한]",76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...",17,2017-08-28 07:09:34.000


In [5]:
train_df['tags_cnt'] = train_df['tags'].map(lambda x : len(x))
train_df['songs_cnt'] = train_df['songs'].map(lambda x : len(x))

In [6]:
def list_merger(x):
    temp_list.extend(x)

temp_list = []

train_df['songs'].map(list_merger)

unique_songs = list(set(temp_list))

num_songs = len(unique_songs)

In [7]:
temp_list = []
                
train_df['tags'].map(list_merger)
                    
unique_tags = list(set(temp_list))
                
num_tags = len(unique_tags)

In [8]:
num_songs

615142

In [9]:
num_tags

29160

In [10]:
num_train = len(train_df)
num_train

115071

### tag에 id부여

In [11]:
train_df.head(3)

,tags,id,plylst_title,songs,like_cnt,updt_date,tags_cnt,songs_cnt
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000,1,19
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000,2,42
2,"[까페, 잔잔한]",76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...",17,2017-08-28 07:09:34.000,2,28


In [12]:
# { 태그 : 새로운id } 딕셔너리
tag_to_id = {}


# { 새로운id : 태그 } 딕셔너리
id_to_tag = {}

# enumerate 반복 횟수를 이용해서 새로운 id 부여
for i,v in enumerate(unique_tags):
    id_to_tag[i] = v

# id_to_tag의 키 밸류를 반전해서 tag_to_id를 만듦
tag_to_id = {v:k for k,v in id_to_tag.items()}


# tags를 id로 변환해서 새로운 column을 생성
def tag_id_trans(x):
    
    temp_list = []
    
    for tag in x:
        temp_list.append(tag_to_id[tag])
    
    return temp_list

train_df['tags_id'] = train_df['tags'].map(tag_id_trans)

In [13]:
train_df.head()

,tags,id,plylst_title,songs,like_cnt,updt_date,tags_cnt,songs_cnt,tags_id
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000,1,19,[3414]
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000,2,42,"[19972, 24204]"
2,"[까페, 잔잔한]",76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...",17,2017-08-28 07:09:34.000,2,28,"[16897, 28271]"
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...",147456,크리스마스 분위기에 흠뻑 취하고 싶을때,"[394031, 195524, 540149, 287984, 440773, 10033...",33,2019-12-05 15:15:18.000,10,38,"[19393, 8057, 11477, 18537, 26347, 19928, 7702..."
4,[댄스],27616,추억의 노래 ㅋ,"[159327, 553610, 5130, 645103, 294435, 100657,...",9,2011-10-25 13:54:56.000,1,53,[14306]


### 플레이리스트 속 노래들(songs)에게 새로운 id 부여

In [14]:
# { 노래id : 새로운id } 딕셔너리
song_to_id = {}


# { 새로운id : 노래id } 딕셔너리
id_to_song = {}

# enumerate 반복 횟수를 이용해서 새로운 id 부여
for i,v in enumerate(unique_songs):
    id_to_song[i] = v

# id_to_tag의 키 밸류를 반전해서 tag_to_id를 만듦
song_to_id = {v:k for k,v in id_to_song.items()}


# tags를 id로 변환해서 새로운 column을 생성
def song_id_trans(x):
    
    temp_list = []
    
    for song in x:
        temp_list.append(song_to_id[song])
    
    return temp_list

train_df['new_songs_id'] = train_df['songs'].map(song_id_trans)

In [15]:
train_df.head(3)

,tags,id,plylst_title,songs,like_cnt,updt_date,tags_cnt,songs_cnt,tags_id,new_songs_id
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000,1,19,[3414],"[456704, 112732, 333158, 488440, 258853, 12127..."
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000,2,42,"[19972, 24204]","[375894, 587314, 431997, 104605, 338568, 21226..."
2,"[까페, 잔잔한]",76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...",17,2017-08-28 07:09:34.000,2,28,"[16897, 28271]","[72132, 240434, 144495, 161861, 307991, 222934..."


### 플레이리스트 새로운 id 부여

In [16]:
train_df['new_plyst_id'] = train_df.index

In [17]:
id_to_playlist = {}

playlist_to_id = {}

def mk_dict(x):
    
    id_to_playlist[x['new_plyst_id']] = x['id']

train_df.loc[:,['id','new_plyst_id']].apply(mk_dict,axis=1)

playlist_to_id = {v:k for k,v in id_to_playlist.items()}

In [18]:
train_df = train_df[['tags','tags_id','id','new_plyst_id','plylst_title','songs','new_songs_id','tags_cnt','songs_cnt','like_cnt','updt_date']]
train_df.head(3)

,tags,tags_id,id,new_plyst_id,plylst_title,songs,new_songs_id,tags_cnt,songs_cnt,like_cnt,updt_date
0,[락],[3414],61281,0,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...","[456704, 112732, 333158, 488440, 258853, 12127...",1,19,71,2013-12-19 18:36:19.000
1,"[추억, 회상]","[19972, 24204]",10532,1,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...","[375894, 587314, 431997, 104605, 338568, 21226...",2,42,1,2014-12-02 16:19:42.000
2,"[까페, 잔잔한]","[16897, 28271]",76951,2,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...","[72132, 240434, 144495, 161861, 307991, 222934...",2,28,17,2017-08-28 07:09:34.000


In [19]:
train_df.columns = ['태그','태그_id','플리_id','새플리_id','플리제목','노래_id','새노래_id','태그수','노래수','좋아요수','갱신일']
train_df.head(3)

,태그,태그_id,플리_id,새플리_id,플리제목,노래_id,새노래_id,태그수,노래수,좋아요수,갱신일
0,[락],[3414],61281,0,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...","[456704, 112732, 333158, 488440, 258853, 12127...",1,19,71,2013-12-19 18:36:19.000
1,"[추억, 회상]","[19972, 24204]",10532,1,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...","[375894, 587314, 431997, 104605, 338568, 21226...",2,42,1,2014-12-02 16:19:42.000
2,"[까페, 잔잔한]","[16897, 28271]",76951,2,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...","[72132, 240434, 144495, 161861, 307991, 222934...",2,28,17,2017-08-28 07:09:34.000


### 변수 정리

- num_songs : 중복 제거된 총 노래수 615142
- num_tags : 중복 제거된 총 태그수 29160
- num_train : 데이터의 총 갯수 115071
 
- tag_to_id : { 태그 : 새로운id } 딕셔너리
- id_to_tag : { 새로운id : 태그 } 딕셔너리
- song_to_id : { 노래 : 새로운id } 딕셔너리
- id_to_song : { 새로운id : 노래 } 딕셔너리
- playlist_to_id { 플리id : 새로운id } 딕셔너리 
- id_to_playlist { 새로운id : 플리id } 딕셔너리 

# 나와 겹치는 플레이리스트를 이용해서 추천 (협업필터링)

## 희소행렬 만들기

- index는 새노래_id이고  column은 새플리_id인 sparse matrix 만들것임!

In [20]:
train_use = train_df[['새플리_id','플리_id','새노래_id','노래_id','노래수']]
train_use.head(3)

,새플리_id,플리_id,새노래_id,노래_id,노래수
0,0,61281,"[456704, 112732, 333158, 488440, 258853, 12127...","[525514, 129701, 383374, 562083, 297861, 13954...",19
1,1,10532,"[375894, 587314, 431997, 104605, 338568, 21226...","[432406, 675945, 497066, 120377, 389529, 24427...",42
2,2,76951,"[72132, 240434, 144495, 161861, 307991, 222934...","[83116, 276692, 166267, 186301, 354465, 256598...",28


In [21]:
#  플레이리스트 별 포함 횟수만큼 반복  * 0번 노래는 19번 나왔으니 0을 19번 찍기
row = np.repeat(range(len(train_use)),train_use['노래수'])

# 플레이 리스트에서 노래 id 를 하나씩 뽑아서 넣어주기
col =  [song for songs in train_use['새노래_id'] for song in songs]

# 나온 노래의 숫자 만큼 1찍기
dat = np.repeat(1, train_use['노래수'].sum())

In [22]:
type(train_use['노래수'])

pandas.core.series.Series

In [23]:
# 세가지를 합해서 sparse csr 매트릭스 생성 , 어떤 플레이리스트에 노래가 있으면 1 없으면 0이 되는 식

# 행이 새노래id, 열이 플리id   (638336 행 115071열)
train_songs_A = spr.csr_matrix((dat, (row, col)), shape=(num_train, num_songs))

# 행이 플리id , 열이 새노래id (115071 행 638336열)
train_songs_A_T = train_songs_A.T.tocsr()

In [36]:
a= []
b =[1,2,3,4]

set(b) - set(a)

{1, 2, 3, 4}

In [23]:
# 오래 걸림 주의!!!
#pd.DataFrame.sparse.from_spmatrix(train_songs_A).head(3)

In [24]:
# 오래 걸림 주의!! 
#pd.DataFrame.sparse.from_spmatrix(train_songs_A_T).head(3)

## 노래를 골라서 하나의 플레이리스트를 만들어보자

In [25]:
# 노래 검색용

with open('data/song_meta.json',encoding='utf-8') as f:
    song_meta = json.load(f)
    
song_meta_df = pd.DataFrame.from_dict(song_meta)
song_meta_df.head(3)

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id
0,[GN0901],20140512,불후의 명곡 - 7080 추억의 얄개시대 팝송베스트,2255639,[2727],Feelings,[GN0900],[Various Artists],0
1,"[GN1601, GN1606]",20080421,"Bach : Partitas Nos. 2, 3 & 4",376431,[29966],"Bach : Partita No. 4 In D Major, BWV 828 - II....",[GN1600],[Murray Perahia],1
2,[GN0901],20180518,Hit,4698747,[3361],Solsbury Hill (Remastered 2002),[GN0900],[Peter Gabriel],2


In [26]:
song_meta_df[song_meta_df['song_name'] == '도망가자 (Run With Me)']

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id
487439,"[GN0401, GN0403]",20191212,Serenade,10364036,[184910],도망가자 (Run With Me),[GN0400],[선우정아],487439


In [27]:
my_playlist = [446812,579156,70549,498312,259414,540535,611737,394686,27469,487439]

my_playlist = sorted(my_playlist)

my_playlist_new_id = []

for song in my_playlist:
    my_playlist_new_id.append(song_to_id[song])
    
my_playlist_new_id

[23775, 61192, 225374, 343065, 388390, 423584, 433070, 469727, 503214, 531542]

In [28]:
song_meta_df.query(f"id == {my_playlist}")

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id
27469,"[GN0401, GN0403]",20160620,Bye bye my blue,2692170,[698776],Bye bye my blue,[GN0400],[백예린],27469
70549,"[GN0105, GN0101]",20190517,너에게 못했던 내 마지막 말은,10286309,[236815],너에게 못했던 내 마지막 말은,[GN0100],[다비치],70549
259414,"[GN2601, GN2603, GN2602]",20070607,Love Child Of The Century,352347,[107725],Lover Boy,[GN2600],[클래지콰이],259414
394686,"[GN0105, GN0101]",20190903,사미서울,10324556,[449861],사미서울,[GN0100],[드레인 (Drain)],394686
446812,"[GN0401, GN0403]",20151130,FRANK,2653573,[698776],우주를 건너,[GN0400],[백예린],446812
487439,"[GN0401, GN0403]",20191212,Serenade,10364036,[184910],도망가자 (Run With Me),[GN0400],[선우정아],487439
498312,"[GN0601, GN0606]",20190321,장범준 3집,10264375,[648551],당신과는 천천히,[GN0600],[장범준],498312
540535,"[GN2601, GN2603]",20040514,Instant Pig,34671,[107725],Gentle Rain,[GN2600],[클래지콰이],540535
579156,"[GN0501, GN0601, GN0503, GN0606, GN0509]",20150604,AUTHENTIC,2322140,[516149],Island Island,"[GN0500, GN0600]",[Bye Bye Badman],579156
611737,"[GN0105, GN2505, GN2501, GN0101, GN2503]",20161101,11:11,10011815,[236797],11:11,"[GN2500, GN0100]",[태연 (TAEYEON)],611737


## 내가 만든 플레이리스트 협업필터링

In [29]:
# 노래의 갯수만큼 0을 가진 1차원 numpy행렬 선언
p = np.zeros((num_songs,1))

# 내가 포함한 노래의 index에서만 1로 바꿈
for music in my_playlist_new_id:
    p[music]=1

# 희소행렬과 dot 연산을 통해 각 플레이리스트 겹치는 노래들의 갯수를 추출
val = train_songs_A.dot(p).reshape(-1)

In [30]:
# 그 플레이리스트에 들어있는 노래들을 추출
cand_song = train_songs_A_T.dot(val)

In [31]:
a = np.array([[1,1,1,0,0,1,0,1,1]]).reshape(3,3)
b = np.array([1,1,0])
a , a.dot(b)

(array([[1, 1, 1],
        [0, 0, 1],
        [0, 1, 1]]),
 array([2, 0, 1]))

In [32]:
print(a.T) # 노래 플리

a.T.dot(a.dot(b))

[[1 0 0]
 [1 0 1]
 [1 1 1]]


array([2, 3, 3])

In [33]:
# 1차원으로 바꿔서 중복된 노래를 가진 플레이리스트 상위 150개 도출
cand_song_idx = cand_song.reshape(-1).argsort()[-150:][::-1]
playlists = cand_song_idx.copy()

songs_already = my_playlist_new_id

# 이미 테스트 데이터가 갖고있는 노래 빼고 100개 도출
cand_song_idx = cand_song_idx[np.isin(cand_song_idx, songs_already) == False][:100]

# 원래 노래 id값으로 복원
rec_song_idx = [id_to_song[i] for i in cand_song_idx]

collabo_result = song_meta_df.query(f"id == {rec_song_idx}")

collabo_result.head(20)

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id
7070,"[GN0805, GN0509, GN0502, GN0801, GN0501]",20170613,남이 될 수 있을까,10070846,"[792022, 714975]",남이 될 수 있을까,"[GN0500, GN0800]","[볼빨간사춘기, 스무살]",7070
15318,"[GN0105, GN0101]",20171218,With Winter,10120289,[726168],With Winter,[GN0100],[민은서밴드],15318
19811,"[GN0105, GN0101]",20191012,새 사랑,10337665,[722098],새 사랑,[GN0100],[송하예],19811
31344,"[GN0105, GN0101]",20191201,헤어진 우리가 지켜야 할 것들,10359196,"[735094, 695994]",헤어진 우리가 지켜야 할 것들,[GN0100],"[김나영, 양다일]",31344
37748,"[GN0105, GN1501, GN0101, GN1504]",20160531,또 오해영 OST Part.5,2687963,[792091],너였다면,"[GN1500, GN0100]",[정승환],37748
41912,"[GN2503, GN2501, GN0101, GN2505]",20190324,사계 (Four Seasons),10265079,[236797],사계 (Four Seasons),"[GN2500, GN0100]",[태연 (TAEYEON)],41912
42155,"[GN0105, GN0101]",20171211,벙어리,10118270,[2018151],벙어리,[GN0100],[홍아],42155
64052,"[GN0805, GN0501, GN0502, GN0801, GN0509]",20160829,Full Album RED PLANET,2707131,[792022],우주를 줄게,"[GN0500, GN0800]",[볼빨간사춘기],64052
65622,"[GN0105, GN0101]",20190620,포장마차,10298014,[1865473],포장마차,[GN0100],[황인욱],65622
68348,"[GN0105, GN0101]",20180306,그리운 기다림..,10144499,[1134286],손편지 (Vocal 태인),[GN0100],[아재],68348
